In [2]:
!pip install parallelPermutationTest
# parallelPermutationTest-1.0.3

import torch
import parallelPermutationTest as ppt
n_bins = 1000
import glob
import numpy as np
import pandas as pd
from google.colab import files
import random



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for parallelPermutationTest: filename=parallelPermutationTest-1.0.3-cp37-cp37m-linux_x86_64.whl size=63297 sha256=5e8eaa7ea38e81542c79edc339ee3385efb0ea0f6cfe3011525b8134777665bd
  Stored in directory: /root/.cache/pip/wheels/13/4a/fd/212f336df03a0d194e345b8f03c7a3bc3bd1bfdbc2eddc4621
Successfully built parallelPermutationTest


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# files containing log2foldchange data
# fdir = 'drive/MyDrive/colab_files/20221003_galvo/'

f_means = glob.glob(fdir+'*_means_*.csv')
f_means

['drive/MyDrive/colab_files/20221003_galvo/20220930_screen_log2fold_diffs_galvo_sgRNA_means_bestruns_3.csv',
 'drive/MyDrive/colab_files/20221003_galvo/20220930_screen_log2fold_diffs_galvo_sgRNA_means_topruns_3.csv',
 'drive/MyDrive/colab_files/20221003_galvo/20220930_screen_log2fold_diffs_galvo_sgRNA_means_3.csv',
 'drive/MyDrive/colab_files/20221003_galvo/20220930_screen_log2fold_diffs_galvo_sgRNA_means_2.csv',
 'drive/MyDrive/colab_files/20221003_galvo/20220930_screen_log2fold_diffs_galvo_sgRNA_means_topruns_2.csv',
 'drive/MyDrive/colab_files/20221003_galvo/20220930_screen_log2fold_diffs_galvo_sgRNA_means_bestruns_2.csv',
 'drive/MyDrive/colab_files/20221003_galvo/20220930_screen_log2fold_diffs_galvo_sgRNA_means_original_analysis.csv',
 'drive/MyDrive/colab_files/20221003_galvo/20220930_screen_log2fold_diffs_galvo_sgRNA_means_topruns_original_analysis.csv',
 'drive/MyDrive/colab_files/20221003_galvo/20220930_screen_log2fold_diffs_galvo_sgRNA_means_bestruns_original_analysis.csv',
 

## Permutation Tests to calculate p-values: Proliferation and Differentiation Screen Datasets



In [7]:
for f in f_means[8:]:
  _df = pd.read_csv(f)
  df_genes_perm = pd.DataFrame()
  print(f)

  df_ctrl = _df[_df.gene.str.contains('CONTROL')]
  control = df_ctrl.log2fold_diff_mean.values 
  control = control[~np.isnan(control)]

  for gene, d in _df[~_df.gene.str.contains('CONTROL')].groupby('gene'):
    pvalue = []
    treatment = d.log2fold_diff_mean.values 

    if len(treatment) < 1:
      df_genes_perm = df_genes_perm.append({'gene' : gene,
                                      'log2fold_diff_mean' : d.log2fold_diff_mean.mean(),
                                      'exp' : f,
                                      'pvalue' : np.nan}, ignore_index=True)

    else:
      while True:
        try:
          pvalue = ppt.GreenFloatCuda(treatment, control, n_bins, batch_size=1)
          break
        except:
          break
      if len(pvalue) == 0:
        pvalue = [np.nan]
      df_genes_perm = df_genes_perm.append({'gene' : gene,
                                            'log2fold_diff_mean' : d.log2fold_diff_mean.mean(),
                                            'exp' : f,
                                            'pvalue' : pvalue[0]}, ignore_index=True)
      
  # Calculate control genes averages using 3 sgRNA per each control 'gene'
  d_ctrl = df_ctrl.copy().reset_index()
  for i in np.arange(2000):
    rndi = [random.randint(0,len(d_ctrl)) for k in np.arange(3)]
    _d = d_ctrl.loc[[random.randint(0,len(d_ctrl)-1) for k in np.arange(3)]] 
    pvalue = ppt.GreenFloatCuda(_d.log2fold_diff_mean.values, control, n_bins, batch_size=1)
    data_list = {'gene' : 'CONTROL_' + str(i),
            'log2fold_diff_mean' : _d.log2fold_diff_mean.mean(),
            'exp' : f,
            'pvalue' : pvalue[0]}
    df_genes_perm = df_genes_perm.append(data_list, ignore_index = True)

  df_genes_perm.to_csv(f.split('.')[0] + '_pvalue.csv')

drive/MyDrive/colab_files/20221003_galvo/20220930_screen_log2fold_diffs_galvo_sgRNA_means_2.csv
drive/MyDrive/colab_files/20221003_galvo/20220930_screen_log2fold_diffs_galvo_sgRNA_means_topruns_original_analysis_pvalue.csv
drive/MyDrive/colab_files/20221003_galvo/20220930_screen_log2fold_diffs_galvo_sgRNA_means_topruns_3_pvalue.csv
drive/MyDrive/colab_files/20221003_galvo/20220930_screen_log2fold_diffs_galvo_sgRNA_means_topruns_2_pvalue.csv
drive/MyDrive/colab_files/20221003_galvo/20220930_screen_log2fold_diffs_galvo_sgRNA_means_original_analysis_pvalue.csv
drive/MyDrive/colab_files/20221003_galvo/20220930_screen_log2fold_diffs_galvo_sgRNA_means_bestruns_original_analysis_pvalue.csv
drive/MyDrive/colab_files/20221003_galvo/20220930_screen_log2fold_diffs_galvo_sgRNA_means_bestruns_3_pvalue.csv


## Lets assess sgRNAs across all the experiments.

I'm aware of a number of genes where  there only appear to be 1/3 of the sgRNA that is effective. However it's a bit unclear who to assess and seperate these from the noise; the ECM data is  a  good example of  this. Lets  take a slightly different and consider each sgRNA across all the  experiments; comparing them to the sgRNA controls. Question: do I compare against all the  control data, or average  each  sgRNA. Maybe try both. The first is probably more correct?

In [5]:
# files containing log2foldchange data
fdir = 'drive/MyDrive/colab_files/20221003_galvo/'
fnames_ = glob.glob(fdir+'*.csv')
fnames = []
for f in fnames_:
  if '_means_' in f:
    continue
  fnames = np.append(fnames,f)

fnames

array(['drive/MyDrive/colab_files/20221003_galvo/20220930_screen_log2fold_diffs_galvo_sgRNA_bestruns_3.csv',
       'drive/MyDrive/colab_files/20221003_galvo/20220930_screen_log2fold_diffs_galvo_sgRNA_3.csv',
       'drive/MyDrive/colab_files/20221003_galvo/20220930_screen_log2fold_diffs_galvo_sgRNA_2.csv',
       'drive/MyDrive/colab_files/20221003_galvo/20220930_screen_log2fold_diffs_galvo_sgRNA_topruns_3.csv',
       'drive/MyDrive/colab_files/20221003_galvo/20220930_screen_log2fold_diffs_galvo_sgRNA_topruns_2.csv',
       'drive/MyDrive/colab_files/20221003_galvo/20220930_screen_log2fold_diffs_galvo_sgRNA_bestruns_2.csv',
       'drive/MyDrive/colab_files/20221003_galvo/20220930_screen_log2fold_diffs_galvo_sgRNA_original_analysis.csv',
       'drive/MyDrive/colab_files/20221003_galvo/20220930_screen_log2fold_diffs_galvo_sgRNA_topruns_original_analysis.csv',
       'drive/MyDrive/colab_files/20221003_galvo/20220930_screen_log2fold_diffs_galvo_sgRNA_bestruns_original_analysis.csv',
 

Now lets go through each sgRNA and compare across ALL experiments (transwell only)

In [10]:
# df = pd.read_csv(fname)
# df = df[df.exp.str.contains('tracketch')]
# print(len(df.sgRNA.unique()))
# df_genes_perm_all_  = pd.read_csv('drive/MyDrive/colab_files/20220809/20211222_screen_log2fold_diffs_tracketchcombined_all_sgRNA_pvalues.csv')
# df = df[~df.sgRNA.isin(df_genes_perm_all_.sgRNA.unique())]
# print(len(df.sgRNA.unique()))

In [ ]:
for f in fnames:
  print(f)
  df_genes_perm_all_ = pd.DataFrame()
  df = pd.read_csv(f)

  control = []
  for sg, d in df[df.annotated_gene_symbol.str.contains('CONTROL')].groupby('sgRNA'):
    control = np.append(control, d['diff'].mean()) 
  control = control[~np.isnan(control)]

  df_ctrl = pd.DataFrame()
  for sg, d in df[df.annotated_gene_symbol.str.contains('CONTROL')].groupby('sgRNA'):
    datalist = {'annotated_gene_symbol' : sg,
                'log2fold_diff_mean' : d['diff'].mean()}
    df_ctrl = df_ctrl.append(datalist, ignore_index = True)  

  # for sg, d in df[~df.gene.str.contains('CONTROL')].groupby('sgRNA'):
  for sg, d in df.groupby('sgRNA'):
    pvalue = None
    
    treatment = d['diff'].values 

    if len(treatment) < 1:
      df_genes_perm_all_ = df_genes_perm_all_.append({'annotated_gene_symbol' :d.gene.unique()[0],
                                                    'sgRNA' : sg,
                                      'log2fold_diff_mean' : d['diff'].mean(),
                                      'exp' : f,
                                      'pvalue' : np.nan}, ignore_index=True)

    else:
      while True:
        try:
          pvalue = ppt.GreenFloatCuda(treatment, control, n_bins, batch_size=1)
          break
        except:
          break
      if len(pvalue) == 0:
        pvalue = [np.nan]
      df_genes_perm_all_ = df_genes_perm_all_.append({'annotated_gene_symbol' :d.annotated_gene_symbol.unique()[0],
                                                    'sgRNA' : sg,
                                            'log2fold_diff_mean' : d['diff'].mean(),
                                            'exp' : f,
                                            'pvalue' : pvalue[0]}, ignore_index=True)
    
    df_genes_perm_all_.to_csv(f.split('.')[0] + '_individualsgRNA_pvalue.csv')

# # Calculate pseudogenes
# d_ctrl = df_ctrl.copy().reset_index()
# for i in np.arange(2000):
#   rndi = [random.randint(0,len(d_ctrl)) for k in np.arange(3)]
#   _d = d_ctrl.loc[[random.randint(0,len(d_ctrl)-1) for k in np.arange(3)]] 
#   pvalue = ppt.GreenFloatCuda(_d.log2fold_diff.values, control, n_bins, batch_size=1)
#   data_list = {'gene' : 'CONTROL_' + str(i),
#                'sgRNA' : np.nan,
#           'log2fold_diff_mean' : _d.log2fold_diff.mean(),
#           'exp' : f,
#           'pvalue' : pvalue[0]}
#   df_genes_perm_all_ = df_genes_perm_all_.append(data_list, ignore_index = True)

    
# df_genes_perm_all_.to_csv('drive/MyDrive/colab_files/20220809/20211222_screen_log2fold_diffs_tracketchcombined_all_sgRNA_pvalues.csv')


drive/MyDrive/colab_files/20221003_galvo/20220930_screen_log2fold_diffs_galvo_sgRNA_bestruns_3.csv


In [1]:
df_genes_perm_all_

NameError: ignored

In [ ]:
# df_genes_perm_all_.to_csv('drive/MyDrive/colab_files/20220809/20211222_screen_log2fold_diffs_tracketchcombined_all_sgRNA_pvalues.csv')
